In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName("pysparkandsql")\
.getOrCreate()

spark

In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS sales")

DataFrame[]

In [4]:
from pyspark.sql.functions import expr
# Customers Data
customers_data = [
(101, 'Ali', 'ali@gmail.com', 'Mumbai', '2022-05-10'),
(102, 'Neha', 'neha@yahoo.com', 'Delhi', '2023-01-15'),
(103, 'Ravi', 'ravi@hotmail.com', 'Bangalore', '2021-11-01'),
(104, 'Sneha', 'sneha@outlook.com', 'Hyderabad', '2020-07-22'),
(105, 'Amit', 'amit@gmail.com', 'Chennai', '2023-03-10'),
]
orders_data = [
(1, 101, 'Laptop', 'Electronics', 2, 50000.0, '2024-01-10'),
(2, 101, 'Mouse', 'Electronics', 1, 1200.0, '2024-01-15'),
(3, 102, 'Tablet', 'Electronics', 1, 20000.0, '2024-02-01'),
(4, 103, 'Bookshelf', 'Furniture', 1, 3500.0, '2024-02-10'),
(5, 104, 'Mixer', 'Appliances', 1, 5000.0, '2024-02-15'),
(6, 105, 'Notebook', 'Stationery', 5, 500.0, '2024-03-01'),
(7, 102, 'Phone', 'Electronics', 1, 30000.0, '2024-03-02'),
]
customers_df = spark.createDataFrame(customers_data, ["CustomerID", "Name", "Email","City", "SignupDate"])
orders_df = spark.createDataFrame(orders_data, ["OrderID", "CustomerID", "Product","Category", "Quantity", "Price", "OrderDate"])
# Write as tables

customers_df.write.mode("overwrite").saveAsTable("sales.customers")
orders_df.write.mode("overwrite").saveAsTable("sales.orders")

In [5]:
# SECTION A: PySpark DataFrame Tasks
# 1. Add a column TotalAmount = Price * Quantity to the orders_df .

orders_df = orders_df.withColumn("TotalAmount",orders_df["Price"]*orders_df["Quantity"])
orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+---------+-----------+--------+-------+----------+-----------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+---------+-----------+--------+-------+----------+-----------+



In [6]:
# 2. Filter all orders with TotalAmount > 10000 .

higher_amount = orders_df.filter(orders_df["TotalAmount"] > 10000)
higher_amount.show()

+-------+----------+-------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+-------+-----------+--------+-------+----------+-----------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      3|       102| Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      7|       102|  Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+-------+-----------+--------+-------+----------+-----------+



In [8]:
# 3. Standardize the City field in customers_df (e.g., lowercase).
from pyspark.sql.functions import lower

customers_df = customers_df.withColumn("City",lower(customers_df["City"]))
customers_df.show()

+----------+-----+-----------------+---------+----------+
|CustomerID| Name|            Email|     City|SignupDate|
+----------+-----+-----------------+---------+----------+
|       101|  Ali|    ali@gmail.com|   mumbai|2022-05-10|
|       102| Neha|   neha@yahoo.com|    delhi|2023-01-15|
|       103| Ravi| ravi@hotmail.com|bangalore|2021-11-01|
|       104|Sneha|sneha@outlook.com|hyderabad|2020-07-22|
|       105| Amit|   amit@gmail.com|  chennai|2023-03-10|
+----------+-----+-----------------+---------+----------+



In [12]:
# 4. Extract year from OrderDate and add a new column OrderYear .
from pyspark.sql.functions import year, to_date

orders_df = orders_df.withColumn("OrderDate",to_date(orders_df["OrderDate"]))
orders_df.show()

orders_df = orders_df.withColumn("OrderYear",year(orders_df["OrderDate"]))
orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+---------+-----------+--------+-------+----------+-----------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+---------+-----------+--------+-------+----------+-----------+

+-------+----------+---------+-----------+--------+-------+----------+-----------+----

In [14]:
# 5. Fill null values in any column of your choice with defaults.
customers_df = customers_df.fillna({"Name":"Name not provided"})
customers_df.show()

+----------+-----+-----------------+---------+----------+
|CustomerID| Name|            Email|     City|SignupDate|
+----------+-----+-----------------+---------+----------+
|       101|  Ali|    ali@gmail.com|   mumbai|2022-05-10|
|       102| Neha|   neha@yahoo.com|    delhi|2023-01-15|
|       103| Ravi| ravi@hotmail.com|bangalore|2021-11-01|
|       104|Sneha|sneha@outlook.com|hyderabad|2020-07-22|
|       105| Amit|   amit@gmail.com|  chennai|2023-03-10|
+----------+-----+-----------------+---------+----------+



In [18]:
# 6. Use when/otherwise to categorize orders:
# <5000 : "Low"
# 5000-20000 : "Medium"
# >20000 : "High"
from pyspark.sql.functions import when

orders_df = orders_df.withColumn("OrderCategory",when(orders_df["TotalAmount"]<5000,"Low")
                                 .when((orders_df["TotalAmount"] >= 5000) & (orders_df["TotalAmount"] <= 20000),"Medium")
                                 .otherwise("High"))
orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|OrderCategory|
+-------+----------+---------+-----------+--------+-------+----------+-----------+---------+-------------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|         High|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|          Low|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|     2024|       Medium|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|     2024|          Low|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|     2024|       Medium|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|     2024|          Low|
|      7|       102|    Phone|Electro

In [20]:
# SECTION B: Spark SQL Tasks
# 7. Run a SQL query to list all orders made by “Ali”.

spark.sql("""select o.orderid,o.category,c.name
from sales.orders o join sales.customers c
on o.customerid = c.customerid
where c.name = 'Ali'""").show()

+-------+-----------+----+
|orderid|   category|name|
+-------+-----------+----+
|      1|Electronics| Ali|
|      2|Electronics| Ali|
+-------+-----------+----+



In [22]:
# 8. Get total spending by each customer using SQL.

spark.sql("""select c.name, sum(o.price * o.quantity) as totalspending
from sales.orders o join sales.customers c
on o.customerid = c.customerid
group by c.name""").show()

+-----+-------------+
| name|totalspending|
+-----+-------------+
| Neha|      50000.0|
|  Ali|     101200.0|
| Ravi|       3500.0|
|Sneha|       5000.0|
| Amit|       2500.0|
+-----+-------------+



In [23]:
# 9. Find out which category made the highest total revenue.

spark.sql("""select category,sum(price*quantity) as Revenuce from sales.orders group by category order by Revenuce desc limit 1""").show()

+-----------+--------+
|   category|Revenuce|
+-----------+--------+
|Electronics|151200.0|
+-----------+--------+



In [25]:
# 10. Create a view customer_orders showing CustomerName, Product, TotalAmount .

spark.sql("""create or replace temp view customer_orders as
select c.name,o.product,o.quantity*o.price as TotalAmount
from sales.orders o join sales.customers c
on o.customerid = c.customerid""")

DataFrame[]

In [28]:
# 11. Query the view for products ordered after Feb 2024.
spark.sql("""select o.product,c.name
from sales.orders o join sales.customers c
on o.customerid = c.customerid where o.OrderDate > '2024-02-29'""").show()

+--------+----+
| product|name|
+--------+----+
|Notebook|Amit|
|   Phone|Neha|
+--------+----+



In [31]:
# SECTION C: Advanced Practice
# 12. Create a Global Temp View from customers_df , then query it using:
# SELECT * FROM global_temp.customers WHERE City = 'Mumbai';

customers_df.createOrReplaceGlobalTempView("customers")

spark.sql("""SELECT * FROM global_temp.customers WHERE City = 'Mumbai'""").show()

+----------+----+-----+----+----------+
|CustomerID|Name|Email|City|SignupDate|
+----------+----+-----+----+----------+
+----------+----+-----+----+----------+



In [33]:
# 13. Save the transformed orders_df (with TotalAmount) to a Parquet file.
orders_df = orders_df.withColumn("TotalAmount",orders_df["Price"]*orders_df["Quantity"])
orders_df.write.mode("overwrite").parquet("/tmp/transformed_orders")

In [34]:
# 14. Read back the Parquet file and count how many orders are in it.
orders_parquet = spark.read.parquet("/tmp/transformed_orders")
print(orders_parquet.count())

7


In [36]:
# SECTION D: UDF + Built-in Function Tasks
# 15. Write a UDF that masks emails like: ali@gmail.com → a***@gmail.com .

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def mask_email(email):
    try:
        local, domain = email.split("@")
        if len(local) > 1:
            return local[0] + "***@" + domain
        else:
            return email
    except Exception:
        return email

mask_email_udf = udf(mask_email, StringType())

customers_df = customers_df.withColumn("MaskedEmail", mask_email_udf("Email"))
customers_df.select("Email", "MaskedEmail").show()

+-----------------+----------------+
|            Email|     MaskedEmail|
+-----------------+----------------+
|    ali@gmail.com|  a***@gmail.com|
|   neha@yahoo.com|  n***@yahoo.com|
| ravi@hotmail.com|r***@hotmail.com|
|sneha@outlook.com|s***@outlook.com|
|   amit@gmail.com|  a***@gmail.com|
+-----------------+----------------+



In [37]:
# 16. Use concat_ws() to create a full label like: 'Ali from Mumbai' .
from pyspark.sql.functions import concat_ws

customers_df = customers_df.withColumn("Label", concat_ws(" from ", customers_df["Name"], customers_df["City"]))
customers_df.select("Name", "City", "Label").show()

+-----+---------+--------------------+
| Name|     City|               Label|
+-----+---------+--------------------+
|  Ali|   mumbai|     Ali from mumbai|
| Neha|    delhi|     Neha from delhi|
| Ravi|bangalore| Ravi from bangalore|
|Sneha|hyderabad|Sneha from hyderabad|
| Amit|  chennai|   Amit from chennai|
+-----+---------+--------------------+



In [38]:
# 17. Use regexp_replace() to remove special characters from product names.
from pyspark.sql.functions import regexp_replace

orders_df = orders_df.withColumn("CleanProduct", regexp_replace("Product", r"[^a-zA-Z0-9 ]", ""))
orders_df.select("Product", "CleanProduct").show()

+---------+------------+
|  Product|CleanProduct|
+---------+------------+
|   Laptop|      Laptop|
|    Mouse|       Mouse|
|   Tablet|      Tablet|
|Bookshelf|   Bookshelf|
|    Mixer|       Mixer|
| Notebook|    Notebook|
|    Phone|       Phone|
+---------+------------+



In [39]:
# 18. Use to_date() and datediff() to calculate customer age in days (from SignupDate to today).
from pyspark.sql.functions import to_date, current_date, datediff

customers_df = customers_df.withColumn("SignupDate", to_date("SignupDate")) \
                           .withColumn("CustomerAgeDays", datediff(current_date(), "SignupDate"))
customers_df.select("Name", "SignupDate", "CustomerAgeDays").show()

+-----+----------+---------------+
| Name|SignupDate|CustomerAgeDays|
+-----+----------+---------------+
|  Ali|2022-05-10|           1121|
| Neha|2023-01-15|            871|
| Ravi|2021-11-01|           1311|
|Sneha|2020-07-22|           1778|
| Amit|2023-03-10|            817|
+-----+----------+---------------+

